In [ ]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --quiet --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 9.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/RuSportSum/train_selector_after_match2.csv" /content/train.csv
!cp "/content/drive/MyDrive/RuSportSum/eval_selector_after_match2.csv" /content/eval.csv

In [ ]:
train = pd.read_csv("/content/train.csv")
eval = pd.read_csv("/content/eval.csv")

data = pd.concat([train, eval])
data['len_content'] = data['content'].apply(lambda x: len(str(x).split()))
data['len_news_name_body'] = data['news_name_body'].apply(lambda x: len(str(x).split()))
data[['len_content', 'len_news_name_body']].describe()

,len_content,len_news_name_body
count,51687.000000,51687.000000
mean,18.191151,59.385687
std,10.875474,26.184353
min,2.000000,14.000000
25%,11.000000,40.000000
50%,16.000000,53.000000
75%,23.000000,72.000000
max,144.000000,192.000000


In [ ]:
temp = train.iloc[:21188]
indexes = temp[temp['relevant_part'] == 0].index.to_list()
print(len(indexes))
train.drop(random.sample(indexes, k=18_000), inplace=True)

20325


In [ ]:
train.to_csv('train.csv', index=False, encoding='utf-8')
eval.to_csv('eval.csv', index=False, encoding='utf-8')

#### Разметка релевантных новостей с активным обучением

In [ ]:
import os
import random
import pandas as pd
from copy import deepcopy

import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [ ]:
class CustomHuggingFaceDatasets(Dataset):
    """
    Support for `huggingface.datasets`: (https://github.com/huggingface/datasets).
    The purpose of this wrapper is to separate the labels from the rest of the sample information
    and make the dataset ready to be used by `baal.active.ActiveLearningDataset`.
    Args:
        dataset (Dataset): a dataset provided by huggingface.
        tokenizer (transformers.PreTrainedTokenizer): a tokenizer provided by huggingface.
        target_key (str): target key used in the dataset's dictionary.
        input_key (str): input key used in the dataset's dictionary.
        max_seq_len (int): max length of a sequence to be used for padding the shorter
            sequences.
    """

    def __init__(
        self,
        dataset,
        tokenizer=None,
        target_key: str = "label",
        input_key_1: str = "sentence1",
        input_key_2: str = "sentence1",
        max_seq_len: int = 200,
    ):
        self.dataset = dataset
        self.targets, self.text1, self.text2 = self.dataset[target_key], self.dataset[input_key_1], self.dataset[input_key_2]
        self.targets_list: List = np.unique(self.targets).tolist()
        self.input_ids, self.attention_masks = (
            self._tokenize(tokenizer, max_seq_len) if tokenizer else ([], [])
        )

    @property
    def num_classes(self):
        return len(self.targets_list)

    def _tokenize(self, tokenizer, max_seq_len):
        # For speed purposes, we should use fast tokenizers here, but that is up to the caller
        tokenized = tokenizer(
            self.text1,
            self.text2,
            add_special_tokens=True,
            max_length=max_seq_len,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",
            truncation=True,
        )
        return tokenized["input_ids"], tokenized["attention_mask"]

    def label(self, idx: int, value: int):
        """Label the item.
        Args:
            idx: index to label
            value: Value to label the index.
        """
        self.targets[idx] = value

    def __len__(self):
        return len(self.text1)

    def __getitem__(self, idx):
        target = self.targets_list.index(self.targets[idx])

        return {
            "input_ids": self.input_ids[idx].flatten() if len(self.input_ids) > 0 else None,
            "inputs": self.text1[idx] + '\n' + self.text2[idx],
            "attention_mask": self.attention_masks[idx].flatten()
            if len(self.attention_masks) > 0
            else None,
            "label": torch.tensor(target, dtype=torch.long),
        }

def custom_active_huggingface_dataset(
    dataset,
    tokenizer=None,
    target_key: str = "label",
    input_key_1: str = "sentence1",
    input_key_2: str = "sentence2",
    max_seq_len: int = 200,
    **kwargs
):
    """
    Wrapping huggingface.datasets with baal.active.ActiveLearningDataset.
    Args:
        dataset (torch.utils.data.Dataset): a dataset provided by huggingface.
        tokenizer (transformers.PreTrainedTokenizer): a tokenizer provided by huggingface.
        target_key (str): target key used in the dataset's dictionary.
        input_key (str): input key used in the dataset's dictionary.
        max_seq_len (int): max length of a sequence to be used for padding the shorter sequences.
        kwargs (Dict): Parameters forwarded to 'ActiveLearningDataset'.
    Returns:
        an baal.active.ActiveLearningDataset object.
    """

    return ActiveLearningDataset(
        CustomHuggingFaceDatasets(dataset, tokenizer, target_key, input_key_1, input_key_2, max_seq_len), **kwargs
    )

Information on the hyperparms below

* epoch: Number of times you want to run and AL loop
* batch_size: The train and eval batch size for hf trainer arguments
* model: Hugging Face Model
* query_size: Number of samples you want to query at each AL iteration for labelling
* heuristic: The acquisition function/heuristic based on which you want to query the important samples
* iterations: The number of iterations you want to run for MCdropout to find the uncertanities
* shuffle_prop: Additional Noise to counter selection bias
* learning_epoch: Traing epochs for hugging face trainer

In [ ]:
hyperparams = {
    "epoch": 4,
    "batch_size": 40,
    "model": "DeepPavlov/rubert-base-cased",
    "query_size": 50,
    "heuristic": "bald",
    "iterations": 10,
    "shuffle_prop": 0.05,
    "learning_epoch": 2,
}

In [ ]:
# Check for CUDA
use_cuda = torch.cuda.is_available()
torch.backends.cudnn.benchmark = True

id2label = {0: "LABEL_0", 1: "LABEL_1"}
label2id = {"LABEL_0": 0, "LABEL_1": 1}

# Load Model
hf_model = AutoModelForSequenceClassification.from_pretrained(
    hyperparams["model"], num_labels=2, id2label=id2label, label2id=label2id
    )

# Setup tokenizer for model
tokenizer = AutoTokenizer.from_pretrained(hyperparams["model"])

# Enable dropouts for predictions
hf_model = patch_module(hf_model)

# Send model to device and setup cuda arguments
if use_cuda:
    hf_model.to("cuda:0")
    no_cuda = False
else:
    hf_model.to("cpu")
    no_cuda = True

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [ ]:
# Define labels in your dataset
label_list = [0, 1]

# Load data from files
data = load_dataset("csv", data_files={'train': '/content/train.csv', 'eval': "/content/eval.csv"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ee9c31cdfe9d6b0a/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def get_label_from_data(active_dataset, data, target, input1, input2, indexes) -> List[int]:
    """
    Get labels from the active dataset, this assumes that you have
    already labelled some samples in your initial dataset

    Args:
    ----
    active_dataset : Active dataset which consists of train and pool

    indexes : Indexes of the points for which labels are to be fetched
    from the data

    Returns:
    ----
    labels: Returns the corresponding labels

    """

    labels = []

    # Now since you labelled points earlier now some part of pool has become train
    # so in order to get the pool indexes based on your 'original' data i.e
    # your raw_train_set. Make sure to user __pool_tp

    raw_data_idx = active_dataset._pool_to_oracle_index(indexes)

    for idx in raw_data_idx:
        #print(f"Adding labels for Raw data Index {idx} : {data[input1][idx]}")
        #print(f"Adding labels for Raw data Index {idx} : {data[input2][idx]}")

        label = data[target][idx]
        labels.append(label)
        #print(label)
        #print("\n")

    return labels

In [ ]:
def get_label_human_oracle(active_dataset, indexes) -> List[int]:
    """
    Get labels from human oracle. During the AL loop some samples
    will go to the human labeller

    Args:
    ----
    active_dataset : Active dataset which consists of train and pool

    indexes : Indexes of the points for which labels are to be fetched
    from the data

    Returns:
    ----
    labels: Returns the corresponding labels

    """
    # List for corresponding labels
    labels = []

    skipped = []

    for sample_idx, idx in enumerate(indexes):

        while True:
            try:
                print(idx)
                pprint(active_dataset.pool.__getitem__(idx)['inputs'], width=150)
                label = int(input())
            except ValueError:
                print("Sorry, I didn't understand that.")
                continue
            if label != -1 and label not in label_list:
                print(f"Allowed labels are {label_list}")
                continue
            if label == -1:
                print("Skipping this sample")
                skipped.append(sample_idx)
                break
            else:
                labels.append(label)
                break
        print("\n")

    indexes_upd = np.delete(indexes, skipped)

    return labels, indexes_upd

In [ ]:
#small_data = data['train'].select(range(10_000))
small_data = data['train']

active_set = custom_active_huggingface_dataset(
    small_data,
    tokenizer=tokenizer,
    target_key='relevant_part',
    input_key_1='content',
    input_key_2='news_name_body'
)

active_set.can_label = True

# 700 первых объектов набора данных
label_from_data = get_label_from_data(active_set, small_data, 'relevant_part', 'content', 'news_name_body', range(3_188))

active_set.label(
    range(3_188),
    label_from_data,
)

valid_set = CustomHuggingFaceDatasets(data['eval'], tokenizer=tokenizer, target_key='relevant_part', input_key_1='content', input_key_2='news_name_body')

active_set, test_set = active_set, valid_set

In [ ]:
# Setup Heuristics
heuristic = get_heuristic(
    hyperparams["heuristic"], hyperparams["shuffle_prop"]
)

# Model save checkpoint
save_checkpoint = 2

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Keep track of initial model weights
init_weights = deepcopy(hf_model.state_dict())

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=hyperparams["learning_epoch"],
    per_device_train_batch_size=hyperparams["batch_size"],
    per_device_eval_batch_size=hyperparams["batch_size"],
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Active Learning Trainer Wrapper
baal_trainer = BaalTransformersTrainer(
    model=hf_model,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)

In [ ]:
logs = []
last_epoch = hyperparams["epoch"] - 1

for epoch in tqdm(range(hyperparams["epoch"])):
    # we use the default setup of HuggingFace for training (ex: epoch=1).
    # The setup is adjustable when BaalHuggingFaceTrainer is defined.
    baal_trainer.train()
    print("\n")

    # Validation!
    #eval_metrics = baal_trainer.evaluate()
    #print("\n")

    if (epoch != last_epoch):
      # MCdropout to gather uncertanities
      predictions = baal_trainer.predict_on_dataset(
          active_set.pool, iterations=hyperparams["iterations"]
      )
      print("\n")

      # Acquistion of the most informative and diverse samples based on BatchBALD
      top_uncertainty = heuristic(predictions)[: hyperparams.get("query_size", 1)]

      # Send the samples for labelling from human oracle
      label_from_oracle, points_to_label_oracle = get_label_human_oracle(
          active_set, top_uncertainty
      )

      # Label active dataset
      active_set.label(points_to_label_oracle, label_from_oracle)

      # We reset the model weights to relearn from the new trainset.
      baal_trainer.load_state_dict(init_weights)
      baal_trainer.lr_scheduler = None

      active_logs = {
          "epoch": epoch,
          "labeled_data": active_set.labelled_map,
          "Next Training set size": len(active_set),
      }
      logs.append({**active_logs})

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.342700,0.286712,0.345613
2,0.209500,0.181042,0.457407




[180-MainThread  ] [baal.transformers_trainer_wrapper:predict_on_dataset_generator:67] 2023-06-23T15:17:10.997215Z [info     ] Start Predict                  dataset=21739



100%|██████████| 544/544 [38:32<00:00,  4.25s/it]




5251
('21 минута. гол! Длинный перевод на Ракитича отрезал половину защиты "Ромы", а тому не составило труда (пусть и с рикошетом от защитника) накинуть '
 'мяч на голову Суаресу - и тот с метра расстрелял ближнюю девятку ворот. Банально, но это гол.\n'
 'Рафинья Алькантара выбыл из строя на полгода из-за разрыва связок колена & Полузащитник «Барселоны» Рафинья Алькантара получил серьезную травму в '
 'матче 1-го группового этапа Лиги чемпионов против «Ромы» (1:1), сообщает tv3. Предварительные тесты показали, что у игрока разрыв связок колена. '
 'Таким образом, Рафинья будет прооперирован. Сроки его восстановления составят минимум 6 месяцев. По информации Mundo Deportivo, сегодня '
 '«Барселона» выступит с официальным заявлением по травме игрока. Напомним, что 22-летний бразильский хавбек повредил колено в столкновении с '
 'полузащитником римлян Раджой Наингголаном . Футболист покинул стадион на костылях. Подробную статистику выступлений Рафиньи смотрите здесь .')
0


13220
('90 м

...


100%|██████████| 541/541 [38:18<00:00,  4.25s/it]




339
('45 минута. Все, завершен первый тайм : ЦСКА забил и ведет в счете 1:0.\n'
 'ЦСКА – «Кубань» – 2:0. Дубль Мусы принес победу армейцам & В 21-м туре чемпионата России в гости к ЦСКА обыграл «Кубань» (2:0). Дублем в этом '
 'матче отметился нападающий армейцев Ахмед Муса. Sports.ru проводил текстовую онлайн-трансляцию матча . Таблица РФПЛ Статистика РФПЛ')
0


13032
('46 минута. 2-е 45.\n'
 'Лига Европы. 1/8 финала. Ответный матч. «Анжи» сыграл вничью с АЗ и выбыл из турнира & Сегодня прошел ответный матч 1/8 финала Лиги Европы. '
 'Российский «Анжи» дома сыграл вничью против АЗ из Алкмара (0:0). Лига Европы 1/8 финала Ответный матч Первый матч: 0:1 Примечание: время начала '
 'матчей – московское. Календарь Лиги Европы Статистика Лиги Европы')
0


14788
('90 минута. +4 минуты\n'
 'Сборная Испании вышла в полуфинал Евро-2012 & Переиграв в матче 1/4 финала Евро-2012 сборную Франции со счетом 2:0, сборная Испании вышла в '
 'следующий раунд турнира. В полуфинале, который состоится 2

 75%|███████▌  | 3/4 [2:42:40<53:29, 3209.43s/it]  

Epoch,Training Loss,Validation Loss,F1
1,0.357800,0.162917,0.471163
2,0.200000,0.207288,0.453405


100%|██████████| 4/4 [2:50:40<00:00, 2560.04s/it]

In [ ]:
m = AutoModelForSequenceClassification.from_pretrained("/content/checkpoint-168")
trainer = BaalTransformersTrainer(
    model=m,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)
output = trainer.evaluate()
output

{'eval_loss': 0.19788295030593872,
 'eval_f1': 0.4738805970149254,
 'eval_runtime': 94.686,
 'eval_samples_per_second': 92.516,
 'eval_steps_per_second': 2.313}

In [ ]:
!cp -r "/content/checkpoint-168" "/content/drive/MyDrive/RuSportSum/bald"